### orm relationships

In [1]:
from sqlalchemy import Column, Integer, String

In [3]:
from sqlalchemy.ext.declarative import declarative_base

In [31]:
Base = declarative_base()

In [39]:
class User(Base):
     __tablename__ = 'users'

     id = Column(Integer, primary_key=True)
     name = Column(String)
     fullname = Column(String)

In [40]:
bob = User(name = 'bob')

In [41]:
bob.name

'bob'

In [42]:
from sqlalchemy.orm import sessionmaker

In [33]:
Session = sessionmaker()

In [47]:
import sqlalchemy
engine = sqlalchemy.create_engine('sqlite:///relationships.db', echo=True)

In [48]:
session = Session(bind=engine)

In [49]:
Base.metadata.create_all(engine)

2018-07-11 15:59:55,871 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-11 15:59:55,873 INFO sqlalchemy.engine.base.Engine ()
2018-07-11 15:59:55,875 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-11 15:59:55,877 INFO sqlalchemy.engine.base.Engine ()
2018-07-11 15:59:55,879 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2018-07-11 15:59:55,881 INFO sqlalchemy.engine.base.Engine ()
2018-07-11 15:59:55,883 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-07-11 15:59:55,885 INFO sqlalchemy.engine.base.Engine ()
2018-07-11 15:59:55,888 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2018-07-11 15:59:55,890 INFO sqlalchemy.engine.base.Engine ()
2018-07-11 15:59:55,898 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-11 15:59:55,900 INFO sqlalchemy.engine.b

In [43]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")

In [44]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

In [51]:
jack = User(name='jack', fullname='Jack Bean')

In [52]:
jack.addresses

[]

In [53]:
jack.addresses = [Address(email_address = 'foobar'), Address(email_address = 'whatnow')]

In [54]:
type(jack.addresses)

sqlalchemy.orm.collections.InstrumentedList

In [55]:
jack.addresses[0].user.name

'jack'

In [58]:
session.query(User).all()

2018-07-11 16:00:27,781 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-11 16:00:27,783 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean')
2018-07-11 16:00:27,787 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2018-07-11 16:00:27,788 INFO sqlalchemy.engine.base.Engine ('foobar', 1)
2018-07-11 16:00:27,791 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2018-07-11 16:00:27,792 INFO sqlalchemy.engine.base.Engine ('whatnow', 1)
2018-07-11 16:00:27,795 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users
2018-07-11 16:00:27,796 INFO sqlalchemy.engine.base.Engine ()


In [57]:
session.add(jack)

In [59]:
session.query(Address).all()

2018-07-11 16:00:54,862 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses
2018-07-11 16:00:54,865 INFO sqlalchemy.engine.base.Engine ()


[<__main__.Address at 0x109a01160>, <__main__.Address at 0x109a011d0>]

In [64]:
jack = session.query(User).filter_by(name = 'jack').first()

2018-07-11 16:36:55,339 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2018-07-11 16:36:55,340 INFO sqlalchemy.engine.base.Engine ('jack', 1, 0)


In [71]:
jack.addresses.sort(key=lambda address: address.email_address)

In [74]:
jack.addresses

[<__main__.Address at 0x109a01160>, <__main__.Address at 0x109a011d0>]

In [83]:
address = jack.addresses[0]
vars(address)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x109a01198>,
 'email_address': 'foobar',
 'id': 1,
 'user': <__main__.User at 0x1099f6cf8>,
 'user_id': 1}

In [88]:
session.query(Address).filter_by(id=1).update({'email_address': 'ok'})

2018-07-11 16:46:01,420 INFO sqlalchemy.engine.base.Engine UPDATE addresses SET email_address=? WHERE addresses.id = ?
2018-07-11 16:46:01,422 INFO sqlalchemy.engine.base.Engine ('ok', 1)


1

In [95]:
session.query(Address).filter_by(id = 2).update({'email_address': 'whaters'})

2018-07-11 16:47:55,520 INFO sqlalchemy.engine.base.Engine UPDATE addresses SET email_address=? WHERE addresses.id = ?
2018-07-11 16:47:55,522 INFO sqlalchemy.engine.base.Engine ('whaters', 2)


1

In [98]:
session.rollback()

2018-07-11 16:48:36,702 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [103]:
session.query(Address).all()

2018-07-11 16:49:20,273 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses
2018-07-11 16:49:20,275 INFO sqlalchemy.engine.base.Engine ()


[]

In [104]:
session.commit()

2018-07-11 16:49:36,474 INFO sqlalchemy.engine.base.Engine COMMIT


### questions

* How to update an existing table using sqlalchemy
* Note that we can do createAll again, and will skip over existing tables